In [ ]:
from dolfin import *
from mshr import *
Lx = 3.0
Ly = 1.0
#resolution = 64
# Create mesh and define function spaces
mesh = RectangleMesh(Point(0.0, 0.0), Point(Lx, Ly), 210, 70, 'right')
#domain = Rectangle(Point(0.0, 0.0), Point(Lx, Ly))
#mesh = generate_mesh(domain, float(resolution))
print("Plotting a RectangleMesh")
plot(mesh, title="Rectangle (left)")

info(mesh)

class Left(SubDomain):
    def inside(self, x, on_boundary):
        return near(x[0], 0.0)

class Right(SubDomain):
    def inside(self, x, on_boundary):
        return near(x[0], Lx)

class Bottom(SubDomain):
    def inside(self, x, on_boundary):
        return near(x[1], 0.0)

class Top(SubDomain):
    def inside(self, x, on_boundary):
        return near(x[1], Ly)

# Sub domain for Periodic boundary condition
class PeriodicBoundary(SubDomain):

    # Left boundary is "target domain" G
    def inside(self, x, on_boundary):
        return bool(x[0] < DOLFIN_EPS and x[0] > -DOLFIN_EPS and on_boundary)

    # Map right boundary (H) to left boundary (G)
    def map(self, x, y):
        y[0] = x[0] - Lx
        y[1] = x[1]

# Initialize sub-domain instances
left = Left()
top = Top()
right = Right()
bottom = Bottom()

# Initialize mesh function for boundary domains
#boundaries = FacetFunction("size_t", mesh) 
boundaries = MeshFunction("size_t", mesh, mesh.topology().dim()-1, 0)
boundaries.set_all(0)
left.mark(boundaries, 1)
top.mark(boundaries, 2)
right.mark(boundaries, 3)
bottom.mark(boundaries, 4)

#plot(boundaries, title = "Boundaries")

#P2 = VectorFunctionSpace(mesh, "CG", 2, constrained_domain=PeriodicBoundary())
#P1 = FunctionSpace(mesh, "CG", 1, constrained_domain=PeriodicBoundary())

P2 = VectorElement("CG", mesh.ufl_cell(), 2)
P1 = FiniteElement("CG", mesh.ufl_cell(), 1)
U0 = FunctionSpace(mesh, P2)
TH = FunctionSpace(mesh, P2*P1, constrained_domain=PeriodicBoundary())

V=FunctionSpace(mesh, "CG", 1, constrained_domain=PeriodicBoundary())

T = TrialFunction(V)
k = TestFunction(V)

temp_hot = Constant(1.0)
temp_pol = Constant(0.0)
temp_init = Constant(0.5)

Ez = Constant((0.0, 1.0))
t_max = 1.0
t = 0
dt = 0.0001
#############################################################################################################################################################
Pr = 0.7
Ra = 1.e7
#############################################################################################################################################################
print("Pr = " + str(Pr) + "\n" + "Ra = " + str(Ra))

u0 = Function(U0)

zero = Constant((0.0, 0.0))
u0 = interpolate(zero, U0)
T0 = interpolate(temp_init, V)
 
(u, p) = TrialFunctions(TH)
(v, q) = TestFunctions(TH)

ds = Measure("ds")[boundaries]

#Velocity field
#a = (1/Pr/dt*inner(u,v) + 1/Pr*inner(grad(u0)*u, v) + inner(grad(u), grad(v)) - div(v)*p + div(u)*q)*dx
#L = (Ra * inner(T0*Ez, v) + 1/Pr*inner(u0,v))*dx

F = (1/Pr/dt*inner(u-u0,v)*dt + 1/Pr*inner(grad(u)*u, v)*dt + inner(grad(u), grad(v))*dt - div(v)*p*dt + div(u)*q*dt)*dx - Ra * inner(T0*Ez, v)*dt*dx

#a = lhs(F)
#L = rhs(F)
#Temperature field
a_T = 1/dt*T*k*dx + inner(grad(T), grad(k))*dx + inner(u0, grad(T))*k*dx
L_T = 1/dt*T0*k*dx

#Boundary conditions
G2 = Expression(("0.0", "0.0"), degree=1)

bc1 = DirichletBC(TH.sub(0), G2, boundaries, 2)
bc2 = DirichletBC(TH.sub(0), G2, boundaries, 4)
bcs1 = [bc1, bc2]

c1 = DirichletBC(V, temp_hot, boundaries, 4)
c2 = DirichletBC(V, temp_pol, boundaries, 2)
bcs2 = [c1, c2]

# Compute solution
l = Function(TH)
T1 = Function(V)
u00 = Function(U0)
# Save solution in VTK format
ufile_pvd = File("./res/velocity.pvd")
pfile_pvd = File("./res/pressure.pvd")
tfile_pvd = File("./res/temperature.pvd")

F = action(F, l)

# define Jacobian
J = derivative(F, l)

# create variational problem and solver
problem = NonlinearVariationalProblem(F, l, bcs1, J)
solver  = NonlinearVariationalSolver(problem)
solver.parameters['newton_solver']['maximum_iterations'] = 25

while t<=t_max:
    # Compute
    begin("Solving ....")
    solver.solve()
    end()
    (u, p) = l.split()
    #assigner = FunctionAssigner(P2, TH.sub(0))
    #assigner.assign(u0, u)
    #u0.assign(u)
    u00 = interpolate(u,U0)
    u0.vector()[:] = u00.vector()       
    #u0 = too   
    #if t%20==0:
    ufile_pvd << u0 
    
    solve(a_T == L_T, T1, bcs2)
    tfile_pvd << T1
    T0.assign(T1)
    
    print(t) 
    t=t+dt

# Plot sigma and u
plot(u, title = "Velocity")
plot(p, title = "Pressure")
plot(T1, title = "Temperature")
interactive()
